#Итоговый проект

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from os import listdir
import codecs
import pickle                                                                   
from bs4 import BeautifulSoup as BS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from boilerpipe.extract import Extractor
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from scipy.sparse import hstack
from scipy.sparse import vstack
from scipy.sparse import hstack
from scipy.sparse import vstack
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
import re
%matplotlib inline

In [2]:
submit = False

In [3]:
def save_obj(obj, name ):                                                       
    with open(name + '.pkl', 'wb') as f:                                        
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            
                                                                                
def load_obj(name ):                                                            
    with open(name + '.pkl', 'rb') as f:                                        
        return pickle.load(f)       

In [4]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return ' '.join(res)

In [5]:
name = 'NumWordsRulesExtractor'
#name = 'ArticleExtractor'
name = 'train_data'+name
train_data = load_obj(name)#Загружаем готовый распарсенный текст уже

In [6]:
#это парсинг теста и доп фич (meta)
if submit:
    test_data= pd.read_csv('test_groups.csv',dtype=np.int16)

    parser_type = 'NumWordsRulesExtractor'
    def save_obj(obj, name ):                                                       
        with open(name + '.pkl', 'wb') as f:                                        
            pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            

    def load_obj(name ):                                                            
        with open(name + '.pkl', 'rb') as f:                                        
            return pickle.load(f)                                                   
        
    description = {}                                                                      
    keywords= {}                                                                      
    test_data['description'] = ''
    test_data['keywords'] = ''

    path = 'content/'                                                               
    texts = {}                                                                      
    print('Start: ' + parser_type)                                                  
    test_data = pd.read_csv('test_groups.csv',dtype=np.int16)                      
    #for filename in ['6770.dat']:                                                  
    for filename in tqdm(listdir(path)):                                            
        doc_id = int(filename.strip('.dat'))                                        
        if doc_id not in test_data.doc_id.values:                                  
            continue                                                                
        try:                                                                        
            with codecs.open(path + filename, 'r', encoding='utf-8', errors='ignore') as f:
                url = f.readline().strip()                                          
                html = f.read()                                                     
                bs = BS(html, 'html.parser')#ищем след страницу                         
                meta = bs.find("meta", {"name":"description"})
                if meta:
                    description[doc_id] = meta.get('content','')
                meta = bs.find("meta", {"name":"keywords"})
                if meta:
                    keywords[doc_id] = meta.get('content','')
                extractor = Extractor(extractor=parser_type, html=html)             
                s = extractor.getText()                                             
                s=s.replace('\n'," ")                                               
                s=s.replace('\t'," ")                                               
                s=s.replace('\r'," ")                                               
                texts[doc_id] = s                                                   
        except UnicodeDecodeError: 
            print('error {}'.format(filename))                                      
            with codecs.open(path + filename, 'r', 'utf-8') as f:                   
                url = f.readline().strip()                                          
                html = f.read()                                                     
            bs = BS(html, 'html.parser')#ищем след страницу                         
            for s in bs.select('script'):                                           
                s.extract()                                                         
            s = bs.get_text()                                                       
            s=s.replace('\n',"")                                                    
            s=s.replace('\t',"")                                                    
            s=s.replace('\r',"")                                                    
            texts[doc_id] = s                                                       
    test_data['text'] = test_data.apply(lambda row: texts[row.doc_id], axis = 1)  
    test_data['description'] = test_data.apply(lambda row: description.get(row.doc_id,''), axis = 1)  
    test_data['keywords'] = test_data.apply(lambda row: keywords.get(row.doc_id,''), axis = 1)  
else:
    path = 'content/'                                                               
    description = {}                                                                      
    keywords= {}                                                                      
    train_data['description'] = ''
    train_data['keywords'] = ''
    #for filename in ['1.dat']:                                                  
    for filename in tqdm(listdir(path)):                                            
        doc_id = int(filename.strip('.dat'))                                        
        if doc_id not in train_data.doc_id.values:                                  
            continue                                                                
        with codecs.open(path + filename, 'r', encoding='utf-8', errors='ignore') as f:
            url = f.readline().strip()                                          
    #        urls[doc_id] = url                                                       
            html = f.read()                                                     
            bs = BS(html, 'html.parser')#ищем след страницу                         
            meta = bs.find("meta", {"name":"description"})
            if meta:
                description[doc_id] = lemmatize(meta.get('content',''))
            meta = bs.find("meta", {"name":"keywords"})
            if meta:
                keywords[doc_id] = lemmatize(meta.get('content',''))
    train_data['description'] = train_data.apply(lambda row: description.get(row.doc_id,''), axis = 1)  
    train_data['keywords'] = train_data.apply(lambda row: keywords.get(row.doc_id,''), axis = 1)  


KeyboardInterrupt: 

In [9]:
len(description),len(keywords)

(8613, 6459)

In [10]:
save_obj(train_data, name + 'lem_meta')

In [45]:
train_data = load_obj(name + 'lem_meta')

In [11]:
#import re
#train_data['urls'] = train_data.apply(lambda row: re.split(';|,|\.|\_|\-|\*|\/|\&|\?',row.urls), axis = 1)  

AttributeError: 'DataFrame' object has no attribute 'urls'

In [47]:
doc_to_title = {}
with open('docs_titles.tsv', encoding = 'UTF-8') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
if submit:
    test_doc_titles = []
    for id in test_data.doc_id.values:
        test_doc_titles.append(doc_to_title[id])
    test_data['title'] = test_doc_titles
    print(len(test_doc_titles))
else:
    doc_titles = []
    for id in train_data.doc_id.values:
        doc_titles.append(doc_to_title[id])
    train_data['title'] = doc_titles
    print(len(doc_titles))


11690


In [92]:
train_data.head(50)

,pair_id,group_id,doc_id,target,text,description,keywords,title,lem_title,sum_title,h1,h2,h3,sum_headers,first_pop,h4,h5,h6
0,1,1,15731,0,Сервисное обслуживание и эксплуатация Отсоедин...,"замена подшипник ступицы, руководство по ремон...","замена подшипник ступицы, технический описание...",ВАЗ 21213 | Замена подшипников ступицы | Нива,ваза 21213 | замена подшипник ступица | нива,ваза 21213 | замена подшипник ступица | ниваза...,16.13 Замена подшипников ступицы,,ВАЗ 21213,16.13 Замена подшипников ступицыВАЗ 21213,0,,,
1,2,1,14829,0,Ваз 2107 оптом в Сочи Смотрите также г. Толья...,"ваза 2107 оптом. продажа, поиск, поставщик и м...","ваза 2107 оптом, россия, сочи, цены, предложен...","Ваз 2107 оптом в Сочи. Сравнить цены, купить п...","ваза 2107 оптом в сочи. сравнить цены, купить ...","ваза 2107 оптом в сочи. сравнить цены, купить ...",\n Ваз 2107 оптом в Сочи\n,,Популярные категории,\n Ваз 2107 оптом в Сочи\nПопулярн...,0,,,
2,3,1,15764,0,Все города ступица Показаны наиболее подходящи...,продажа запчасть ступица для легковой и грузов...,"цена, замена, тюнинг, купить, продать, ступица...",Купить ступица Лада калина2. Трансмиссия - пер...,купить ступица лада калина2. трансмиссия - пер...,купить ступица лада калина2. трансмиссия - пер...,\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\tСтупица Лада...,,Запчасть,\t\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\tСтупица Лада...,0,,,
3,4,1,17669,0,"Классика 21010 - 21074 121,00 121,00 539,00 57...",,,Классика 21010 - 21074,классика 21010 - 21074,классика 21010 - 21074,Learn Center,Классика 21010 - 21074Прайс,,Learn CenterКлассика 21010 - 21074Прайс,0,,,
4,5,1,14852,0,Содержание статьи: Ступица Нива переднего коле...,передний ступица нива иметь свой особенность в...,,Ступица Нива — замена подшипника своими руками,ступица нива — замена подшипник свой рука,ступица нива — замена подшипник свой рукаперед...,Ступица Нива — как провести ремонт и замену по...,,Устройство и особенности cтупицы переднего кол...,Ступица Нива — как провести ремонт и замену по...,0,,,
5,6,1,15458,0,Проверка модуля зажигания ВАЗ 2110 мультиметро...,"если менять и пружины, то наиболее распростран...",,ВАЗ 2110,ваза 2110,"ваза 2110если менять и пружины, то наиболее ра...",ВАЗ 2110,Проверка модуля зажигания ВАЗ 2110 мультиметро...,,ВАЗ 2110Проверка модуля зажигания ВАЗ 2110 мул...,0,,,
6,7,1,14899,0,форум: http://forum.teoria-dvs.com/index.php ...,форум http forum.teoria dvs.com index.php груп...,"21212123, ваз, полуоси, подшипников, обзор, ob...","Обзор подшипников полуоси ВАЗ 2101-07, 2121,2123","обзор подшипник полуось ваза 2101-07, 2121,2123","обзор подшипник полуось ваза 2101-07, 2121,212...","Обзор подшипников полуоси ВАЗ 2101-07, 2121,2123",,,"Обзор подшипников полуоси ВАЗ 2101-07, 2121,2123",0,,,
7,8,1,16879,0,Название: BMW X5 (E70) | X6 (E71) VOLVO 850 (L...,страница 23: подшипник и ступица fag с доставк...,"купить подшипники, ступица fag в киеве, харько...",Купить Подшипники и ступицы FAG (Страница 23),купить подшипник и ступица fag (страница 23),купить подшипник и ступица fag (страница 23)ст...,FAG,Ступичный подшипник FAG 713 6494 50Ступичный п...,,FAGСтупичный подшипник FAG 713 6494 50Ступичны...,0,Запчасти,,
8,9,1,16310,0,Отзыв владельца ВАЗ 2121 Нива 2007 года Как и ...,,,HorsePowers — автомобильный интернет портал » ...,horsepowers — автомобильный интернет портал » ...,horsepowers — автомобильный интернет портал » ...,Отзыв владельца ВАЗ 2121 Нива 2007 года,,Авто новинкиИнтересноеАвтоновостиТест-драйвыВи...,Отзыв владельца ВАЗ 2121 Нива 2007 годаАвто но...,0,LADA 4×4 Нива (ВАЗ-2121 и ВАЗ-2131),,
9,10,1,15440,0,Закроется через 7 час. 43 мин. Моя оценка В пр...,новость и сообщение из официальный группа вкон...,"торговый компания 33 sport группа вконтакте, т...",Новости и сообщения из официальной группы Вкон...,новость и сообщение из официальный группа вкон...,новость и сообщение из официальный группа вкон...,\nОфициальная группа Вконтакте торговой компан...,\r\n\t\t\t\t\t\tТорговая компания 33 Sport\t\t...,Магазины на Ботанической ул

In [11]:
def text_cleaning(data):
    import string
    import unicodedata
    exclude = set(string.punctuation)
    exclude.add('…')
    clear_text = []
    for text in tqdm(data.text.values):
        #newtext = text.replace(u'\xa0', u' ')
        #newtext = text.replace(u'\xad', u' ')
        newtext = unicodedata.normalize("NFKD",text)
        #newtext = re.sub(r'\d+', '', newtext)

        for punctuation in exclude:
            newtext = newtext.replace(punctuation," ")
            
        clear_text.append(newtext)
        
    return clear_text

In [49]:
if submit:
    test_data['lem_title'] = test_data.apply(lambda row:lemmatize(row.title) , axis = 1)  
else:
    train_data['lem_title'] = train_data.apply(lambda row:lemmatize(row.title) , axis = 1)  

In [50]:
if submit:
    test_data['sum_title'] = test_data.apply(lambda row: row.lem_title +row.description + row.keywords , axis = 1)  
else:
    train_data['sum_title'] = train_data.apply(lambda row: row.lem_title +row.description + row.keywords , axis = 1)  


In [95]:
path = 'content/'                                                                                                                                   
divs = {}
h1s = {}
h2s = {}
h3s = {}
h4s = {}
h5s = {}
h6s = {}
for filename in tqdm(listdir(path)):                                            
    doc_id = int(filename.strip('.dat'))                                        
    if doc_id not in train_data.doc_id.values:                                  
        continue                                                                
    with codecs.open(path + filename, 'r', encoding='utf-8', errors='ignore') as f:
        html = f.read()                                                     
        bs = BS(html, 'html.parser')#ищем след страницу                         
        h1 = bs.find_all("h1")
        h2 = bs.find_all("h2")
        h3 = bs.find_all("h3")
        h4 = bs.find_all("h4")
        h5 = bs.find_all("h5")
        h6 = bs.find_all("h6")
        str = ''
        for h in h1:
            s=h.text.replace('\n'," ")                                                    
            s=s.replace('\t'," ")                                                    
            s=s.replace('\r'," ")          
            str+=' '+s
        h1s[doc_id] = str
        
        str = ''
        for h in h2:
            s=h.text.replace('\n'," ")                                                    
            s=s.replace('\t'," ")                                                    
            s=s.replace('\r'," ")          
            str+=' '+s        
        h2s[doc_id] = str
        
        str = ''
        for h in h3:
            s=h.text.replace('\n'," ")                                                    
            s=s.replace('\t'," ")                                                    
            s=s.replace('\r'," ")          
            str+=' '+s
        h3s[doc_id] = str
        
        
        str = ''
        for h in h4:
            s=h.text.replace('\n'," ")                                                    
            s=s.replace('\t'," ")                                                    
            s=s.replace('\r'," ")          
            str+=' '+s 
        h4s[doc_id] = str
        
        str = ''
        for h in h5:
            s=h.text.replace('\n'," ")                                                    
            s=s.replace('\t'," ")                                                    
            s=s.replace('\r'," ")          
            str+=' '+s
        h5s[doc_id] = str
        
        str = ''
        for h in h6:
            s=h.text.replace('\n'," ")                                                    
            s=s.replace('\t'," ")                                                    
            s=s.replace('\r'," ")          
            str+=' '+s
        h6s[doc_id] = str
        
train_data['h1'] = train_data.apply(lambda row: h1s.get(row.doc_id,''), axis = 1)  
train_data['h2'] = train_data.apply(lambda row: h2s.get(row.doc_id,''), axis = 1)  
train_data['h3'] = train_data.apply(lambda row: h3s.get(row.doc_id,''), axis = 1)  
train_data['h4'] = train_data.apply(lambda row: h4s.get(row.doc_id,''), axis = 1)  
train_data['h5'] = train_data.apply(lambda row: h5s.get(row.doc_id,''), axis = 1)  
train_data['h6'] = train_data.apply(lambda row: h6s.get(row.doc_id,''), axis = 1)  


In [98]:
train_data

,pair_id,group_id,doc_id,target,text,description,keywords,title,lem_title,sum_title,h1,h2,h3,sum_headers,first_pop,h4,h5,h6
0,1,1,15731,0,Сервисное обслуживание и эксплуатация Отсоедин...,"замена подшипник ступицы, руководство по ремон...","замена подшипник ступицы, технический описание...",ВАЗ 21213 | Замена подшипников ступицы | Нива,ваза 21213 | замена подшипник ступица | нива,ваза 21213 | замена подшипник ступица | ниваза...,16.13 Замена подшипников ступицы,,ВАЗ 21213,16.13 Замена подшипников ступицы ВАЗ 21213,0,,,
1,2,1,14829,0,Ваз 2107 оптом в Сочи Смотрите также г. Толья...,"ваза 2107 оптом. продажа, поиск, поставщик и м...","ваза 2107 оптом, россия, сочи, цены, предложен...","Ваз 2107 оптом в Сочи. Сравнить цены, купить п...","ваза 2107 оптом в сочи. сравнить цены, купить ...","ваза 2107 оптом в сочи. сравнить цены, купить ...",Ваз 2107 оптом в Сочи,,Популярные категории,Ваз 2107 оптом в Сочи Популярн...,0,,,
2,3,1,15764,0,Все города ступица Показаны наиболее подходящи...,продажа запчасть ступица для легковой и грузов...,"цена, замена, тюнинг, купить, продать, ступица...",Купить ступица Лада калина2. Трансмиссия - пер...,купить ступица лада калина2. трансмиссия - пер...,купить ступица лада калина2. трансмиссия - пер...,Ступица Лада калина2,,Запчасть,Ступица Лада калина2 ...,0,,,
3,4,1,17669,0,"Классика 21010 - 21074 121,00 121,00 539,00 57...",,,Классика 21010 - 21074,классика 21010 - 21074,классика 21010 - 21074,Learn Center,Классика 21010 - 21074 Прайс,,Learn Center Классика 21010 - 21074 Прайс,0,,,
4,5,1,14852,0,Содержание статьи: Ступица Нива переднего коле...,передний ступица нива иметь свой особенность в...,,Ступица Нива — замена подшипника своими руками,ступица нива — замена подшипник свой рука,ступица нива — замена подшипник свой рукаперед...,Ступица Нива — как провести ремонт и замену п...,,Устройство и особенности cтупицы переднего ко...,Ступица Нива — как провести ремонт и замену п...,0,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11685,11686,129,26672,0,"25.03.2014 Котейка Мурысовна, мальчик Миша 335...",,,❤★✿★АПРЕЛЯТА 2014 -6❤★✿,❤★✿★апрелёнок 2014 -6❤★✿,❤★✿★апрелёнок 2014 -6❤★✿,,,❤★✿★АПРЕЛЯТА 2014 -6❤★✿,❤★✿★АПРЕЛЯТА 2014 -6❤★✿,0,,,
11686,11687,129,25838,0,Geschrieben von Romana am 05.05.2012 Schöne Ho...,,,:: Gästebuch,:: gästebuch,:: gästebuch,,links:,,links: Wir freuen uns über Ihren Kommentar An...,0,Wir freuen uns über Ihren Kommentar,Antwort schreiben,
11687,11688,129,25703,0,"РЕБЯЯЯТА!! / ЗАХОДИМ И ПОМОГАЕМ!, Реферат ф фи...",,,Jizolofej: Archive,jizolofej: archive,jizolofej: archive,Jizolofej Философия ф бэкона реферат Кaртошкa...,"April 2014 РЕБЯЯЯТА!! / ЗАХОДИМ И ПОМОГАЕМ!, ...",Готовимся к chgk Таганрог (СКЦ “Приморский”) ...,Jizolofej Философия ф бэкона реферат Кaртошкa...,0,,,
11688,11689,129,27885,0,1 апреля прошлого года 21-летний Сергей Семе...,,,Как зовут парня дианы шурыгиной | Пусть говоря...,как звать парень диана шурыгин | пусть говорят...,как звать парень диана шурыгин | пусть говорят...,Как зовут парня дианы шурыгиной,Пусть говорят - В разгар вечеринки. Часть 1. ...,,Как зовут парня дианы шурыгиной Пусть говорят...,0,,,


In [97]:
train_data['sum_headers'] = train_data.apply(lambda row: row.h1 +row.h2 + row.h3 +row.h4 + row.h5 + row.h6, axis = 1)  


In [107]:
train_data['lem_headers'] = train_data.apply(lambda row:lemmatize(row.sum_headers) , axis = 1)  


In [ ]:
# from tqdm.notebook import tqdm
# tqdm.pandas()

# # Now you can use `progress_apply` instead of `apply`

In [ ]:
#train_data['text'] = train_data.progress_apply(lambda row:lemmatize(row.text[0:1000]) , axis = 1)  
# train_data['text'] = train_data.progress_apply(lambda row: row.text[0:len(row.title)] , axis = 1)  

In [ ]:

# from tqdm.notebook import tqdm
# tqdm.pandas()
# train_data['text'] = train_data.progress_apply(lambda row: re.sub(r'\d+', '', row.text) , axis = 1)  

In [99]:
def get_splits(data, n=3, random_state=None):
    groups = train_data.group_id.unique()
    kf = KFold(n_splits=n, shuffle=True, random_state=random_state)
    for i,j in kf.split(groups):
        train, test = data.group_id.isin(groups[i].reshape(-1)), data.group_id.isin(groups[j].reshape(-1))
        yield (train[train].index.values, test[test].index.values)

In [100]:
#vectorizer = CountVectorizer(ngram_range = (1,3))#max_features = 70)
#stop_words =['без','в','вне','во','для','до','за','И','из','изо','к','ко','меж','на','над','о','об','обо','от','ото','по','под','при','про','с', 'а','же','и','ибо','или','как','но','раз','чем','что','не','ни','ли','ль','вот','вон','это','то','ну','как','же','ни','уж','ну','я', 'ты', 'он', 'мы', 'вы', 'её', 'их','кто','тот']

 
vectorizer = CountVectorizer()#max_features = 70)
#vectorizer = TfidfVectorizer()#(stop_words=stop_words)#max_features = 70)

In [101]:
all_groups = np.array(list(range(1,310))).reshape(-1,1)
i = OneHotEncoder()
i.fit(all_groups)
if submit:
    groups_one_hot_test   = i.transform(test_data.group_id.values.reshape(-1,1))
else:
    groups_one_hot_train = i.transform(train_data.group_id.values.reshape(-1,1))

In [20]:
def get_top_n_words(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.

    get_top_n_words(["I love Python", "Python is a language programming", "Hello world", "I love the world"]) -> 
    [('python', 2),
     ('world', 2),
     ('love', 2),
     ('hello', 1),
     ('is', 1),
     ('programming', 1),
     ('the', 1),
     ('language', 1)]
    """
    vec = CountVectorizer(stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

    return [i[0] for i in words_freq[:n]]

In [114]:
def get_features(data):
    scaler = StandardScaler(with_mean=False)
    number = 1
    data['first_pop'] =0
    for group in tqdm(data.group_id.unique()):
        number +=1
        elements_in_same_group = data.group_id == group
        #words = data.title[elements_in_same_group] + data.text[elements_in_same_group]
        words = data.lem_title[elements_in_same_group]
        vectorizer.fit(words)
        texts_vectorized_titles = vectorizer.transform(data.lem_title[elements_in_same_group])
        texts_vectorized_titles = scaler.fit_transform(texts_vectorized_titles)

        texts_vectorized_texts = vectorizer.transform(data.text[elements_in_same_group])
        texts_vectorized_texts = scaler.fit_transform(texts_vectorized_texts)
        texts_vectorized_keywords = vectorizer.transform(data.keywords[elements_in_same_group])
        texts_vectorized_keywords = scaler.fit_transform(texts_vectorized_keywords)
        texts_vectorized_description= vectorizer.transform(data.description[elements_in_same_group])
        texts_vectorized_description= scaler.fit_transform(texts_vectorized_description)
    
        texts_vectorized_sumtitles = vectorizer.transform(data.sum_title[elements_in_same_group])
        texts_vectorized_sumtitles = scaler.fit_transform(texts_vectorized_sumtitles)
        
        texts_vectorized_headers = vectorizer.transform(data.sum_headers[elements_in_same_group])
        texts_vectorized_headers = scaler.fit_transform(texts_vectorized_headers)
        
        dist_matr = pairwise_distances(texts_vectorized_titles, texts_vectorized_titles, metric = 'cosine')
        dist_matr2 = pairwise_distances(texts_vectorized_texts, texts_vectorized_texts, metric = 'cosine')
        dist_matr3 = pairwise_distances(texts_vectorized_texts, texts_vectorized_titles, metric = 'cosine')
        dist_matr4 = pairwise_distances(texts_vectorized_keywords, texts_vectorized_keywords, metric = 'cosine')
        dist_matr5 = pairwise_distances(texts_vectorized_description, texts_vectorized_description, metric = 'cosine')
        dist_matr6 = pairwise_distances(texts_vectorized_sumtitles, texts_vectorized_texts, metric = 'cosine')
        dist_matr7 = pairwise_distances(texts_vectorized_keywords, texts_vectorized_titles, metric = 'cosine')
        dist_matr8 = pairwise_distances(texts_vectorized_description, texts_vectorized_titles, metric = 'cosine')
        dist_matr9 = pairwise_distances(texts_vectorized_headers, texts_vectorized_headers, metric = 'cosine')
        #dist_matr10 = pairwise_distances(texts_vectorized_headers, texts_vectorized_titles, metric = 'cosine')

        #clustering = DBSCAN(eps = 1.3, min_samples=5).fit(dist_matr)
        #clf = IsolationForest(random_state=0).fit_predict(dist_matr)

        dist_matr_sorted = np.sort(dist_matr,axis=1)[:,0:21]
        dist_matr_sorted2 = np.sort(dist_matr2,axis=1)[:,0:21]
        dist_matr_sorted3 = np.sort(dist_matr3,axis=1)[:,0:21]
        dist_matr_sorted4 = np.sort(dist_matr4,axis=1)[:,:-1][:,0:21]
        dist_matr_sorted5 = np.sort(dist_matr5,axis=1)[:,:-1][:,0:21]
        dist_matr_sorted6 = np.sort(dist_matr6,axis=1)[:,:-1][:,0:21]
        dist_matr_sorted7 = np.sort(dist_matr7,axis=1)[:,:-1][:,0:21]
        dist_matr_sorted8 = np.sort(dist_matr8,axis=1)[:,:-1][:,0:21]
        dist_matr_sorted9 = np.sort(dist_matr9,axis=1)[:,:-1][:,0:21]
        #dist_matr_sorted10 = np.sort(dist_matr9,axis=1)[:,:-1][:,0:21]

#         dist_matr_sorted = np.sort(dist_matr,axis=1)[:,0:min(50,dist_matr.shape[1])]
#         dist_matr_sorted2 = np.sort(dist_matr2,axis=1)[:,0:min(50,dist_matr2.shape[1])]
#         dist_matr_sorted3 = np.sort(dist_matr3,axis=1)[:,0:min(50,dist_matr3.shape[1])]

#         if dist_matr_sorted.shape[1] < 50:
#             dist_matr_sorted = np.hstack((dist_matr_sorted, np.tile(dist_matr_sorted[:,-1:], (1, 50-dist_matr_sorted.shape[1]))))
#             dist_matr_sorted2 = np.hstack((dist_matr_sorted2, np.tile(dist_matr_sorted2[:,-1:], (1, 50-dist_matr_sorted2.shape[1]))))
#             dist_matr_sorted3 = np.hstack((dist_matr_sorted3, np.tile(dist_matr_sorted3[:,-1:], (1, 50-dist_matr_sorted3.shape[1]))))


        mean_dist = dist_matr.mean(axis = 1)
        mean_dist2 = dist_matr2.mean(axis = 1)
        mean_dist3 = dist_matr3.mean(axis = 1)
        mean_dist4 = dist_matr4.mean(axis = 1)
        mean_dist5 = dist_matr5.mean(axis = 1)
        mean_dist6 = dist_matr6.mean(axis = 1)
        mean_dist7 = dist_matr7.mean(axis = 1)
        mean_dist8 = dist_matr8.mean(axis = 1)
        mean_dist9 = dist_matr9.mean(axis = 1)
        #mean_dist10 = dist_matr10.mean(axis = 1)
        
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist2.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist3.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist4.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist5.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist6.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist7.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist8.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist9.reshape(-1,1)))
        #dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist10.reshape(-1,1)))

#         mean_dist = mean_dist *mean_dist 
#         mean_dist2 = mean_dist2 *mean_dist2 
#         mean_dist3 = mean_dist3 *mean_dist3 
#         mean_dist4 = mean_dist4 *mean_dist4 
#         mean_dist5 = mean_dist5 *mean_dist5 
#         mean_dist6 = mean_dist6 *mean_dist6 
#         mean_dist7 = mean_dist7 *mean_dist7 
#         dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist.reshape(-1,1)))
#         dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist2.reshape(-1,1)))
#         dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist3.reshape(-1,1)))
#         dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist4.reshape(-1,1)))
#         dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist5.reshape(-1,1)))
#         dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist6.reshape(-1,1)))
#         dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist7.reshape(-1,1)))
        
#         pop_words = get_top_n_words(words,n=100)
#         for pop_word in pop_words:
#             data.loc[elements_in_same_group,'first_pop']= data[elements_in_same_group].apply(lambda row: int(pop_word in row.text) , axis = 1)  
#             pop  = csr_matrix(data[elements_in_same_group].first_pop.values)
#             print(pop_word,pop.toarray())
#             dist_matr_sorted = np.hstack((dist_matr_sorted, pop.toarray().reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted2))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted3))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted4))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted5))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted6))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted7))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted8))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted9))
        #dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted10))
#        dist_matr_sorted = np.hstack((dist_matr_sorted, clustering.labels_.reshape(-1,1)))
#        dist_matr_sorted = hstack((dist_matr_sorted, texts_vectorized))

        if number == 2 :
            features_by_dist =  dist_matr_sorted
        else:
            features_by_dist = vstack((features_by_dist, dist_matr_sorted))
    return features_by_dist.tocsr()

In [115]:
if submit:
    features_by_dist = get_features(test_data)
    features_by_dist = hstack([features_by_dist, groups_one_hot_test])
    features_by_dist = features_by_dist.tocsr()
else:
    features_by_dist = get_features(train_data)
    features_by_dist = hstack([features_by_dist, groups_one_hot_train])
    features_by_dist = features_by_dist.tocsr()

In [19]:
features_by_dist.shape

(11690, 485)

In [105]:
target = train_data.target.values

In [30]:
a = np.array([0.015, 0.535, 0.112])
a[[1,2]] = [0,5]
a

array([0.015, 0.   , 5.   ])

In [116]:
#VALIDATION
import xgboost
max_score = 0
scores = []
j = features_by_dist
for i in tqdm(range(5)):
    for train, test in get_splits(train_data,n=3,random_state=i):
         #model = LogisticRegression(n_jobs=-1)
        model = RandomForestClassifier(n_jobs=6,n_estimators=80)
#        model = GradientBoostingClassifier(n_estimators=2000, learning_rate=0.002, max_depth=9)
#        model = LogisticRegression(n_jobs=6)pca = PCA(n_components=2)
#        model.fit(features_by_dist[train], target[train])
        model.fit(j[train], target[train])
        pred = model.predict(j[test])
    #new_j[test] = pred
        #model2.fit(j[train], target[train])
        #pred2 = model2.predict(j[test]) 
        score = f1_score(pred,train_data.target[test])
        scores.append(score)
        print(score, end=' ')
    print()

print('\nAvg score:' ,sum(scores)/len(scores))

0.7549441100601892 0.7622571692876967 0.7344689378757515 
0.7274481427882296 0.7516198704103672 0.7375 
0.784219001610306 0.7626582278481013 0.683247301736274 
0.7849162011173186 0.736886277801091 0.7164790174002047 
0.7189716312056738 0.7569872098531502 0.774582560296846 


Avg score: 0.7458123772860801


In [37]:
new_j.shape

(11690, 486)

In [ ]:
#SAVE model
from sklearn.ensemble import RandomForestClassifier 
model = RandomForestClassifier(n_jobs=6,n_estimators=1000)
model.fit(features_by_dist, target)
save_obj(model, 'model')

In [29]:
#SUBMIT
from sklearn.ensemble import RandomForestClassifier 
print(type(features_by_dist))
model = load_obj('model') 
pred = model.predict(features_by_dist)               
pred.shape

<class 'scipy.sparse.csr.csr_matrix'>


(16627,)

In [30]:
test_data['target'] = pred
test_data.to_csv('sub_with_meta',index=False,columns=['pair_id','target'])

In [ ]:
from time import sleep
gr = int(input())
print(gr)
pca = PCA(n_components=components=2)
i = pca.fit_transform(features_by_dist[train_data[train_data.group_id==gr].index.values].toarray())
a = plt.subplot()
a.scatter(i[:, 0], i[:, 1], c=target[train_data[train_data.group_id==gr].index.values])

In [ ]:
save_obj(test_data,'test')

In [ ]:
# train_data['urls'] = train_data.apply(lambda row:lemmatize(row.urls) , axis = 1)  